# Todays News Trends

### Program gets the current to headlines from 
* All News Media *
### and performs sentiment analysis on the news to identify the current sentiment.


In [26]:
import tweepy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import time

import requests

from pprint import pprint

%matplotlib inline

# News API
import newsapi
from newsapi.newsapi_client import NewsApiClient
from newsapi.articles import Articles
from newsapi.sources import Sources

# NLTK
import nltk
from nltk import sent_tokenize
from string import punctuation

In [11]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

#from config import (consumer_key, consumer_secret,
#                    access_token, access_token_secret)

# Twitter API Keys
consumer_key = "lQDyO9LHsHV7rw12KbBGP2H4S"
consumer_secret = "SrjdhJcNqJUsnMckBdp9vICDSXS8udXvEJr1iZQKKgMmyp4rsS"
access_token = "1007989024773074944-XZqI1XqtOjXdGDZlEMlftuNkRQbCVe"
access_token_secret = "58vmGsdNdKv7dXmc1NfMs7VfnbGarNgelr4IGnQf6bLjj"

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

#News API
#NewsAPI
newapi_key = '4b7e7bb7670e40f788f92667a6a2dd64'
# Init
newsapi = NewsApiClient(api_key='4b7e7bb7670e40f788f92667a6a2dd64')
newsapi_Articles = Articles(API_KEY=newapi_key)
newsapi_Sources = Sources(API_KEY=newapi_key)

In [13]:
# Get the top news newsapi, in the US, english, all categories.
def getCurrentHeadlines(sources='ALL'):
    top_headlines = newsapi.get_top_headlines(q=None, 
                                              sources=None, 
                                              language='en', 
                                              country=None, 
                                              category=None, 
                                              page_size=25, 
                                              page=None)
    
    #print(f"Total Results {top_headlines['totalResults']}")
    return top_headlines

In [15]:
#titles = [article['title'] for article in top_headlines['articles']]
headlines = getCurrentHeadlines()
#pprint(headlines)

In [16]:
len(headlines['articles'])

25

In [17]:
news_top_headlines_df = pd.DataFrame.from_dict(headlines['articles'])

In [19]:
news_top_headlines_df

,author,description,publishedAt,source,title,url,urlToImage
0,Yonah Jeremy Bob,HRW lawyer: Government not really after “the ‘...,2018-06-28T03:12:00Z,"{'id': 'the-jerusalem-post', 'name': 'The Jeru...",To deport HRW official or not? Court wrestles ...,https://www.jpost.com/Israel-News/To-deport-Hu...,"https://images.jpost.com/image/upload/f_auto,f..."
1,Tovah Lazaroff,'As long as the politics and the obsessive pur...,2018-06-28T02:20:00Z,"{'id': 'the-jerusalem-post', 'name': 'The Jeru...",Israeli envoy: We should delay UNESCO exit,https://www.jpost.com/Arab-Israeli-Conflict/Is...,"https://images.jpost.com/image/upload/f_auto,f..."
2,Fox News,President Trump praised retiring Supreme Court...,2018-06-28T01:15:00+00:00,"{'id': 'fox-news', 'name': 'Fox News'}",Trump says keeping Senate in GOP hands 'the mo...,http://www.foxnews.com/politics/2018/06/27/tru...,http://a57.foxnews.com/media2.foxnews.com/Brig...
3,Reuters,An application for a warrant to search Paul Ma...,2018-06-28T01:07:57.9889393Z,"{'id': 'nbc-news', 'name': 'NBC News'}",Russian oligarch loaned $10 million to Manafor...,http://www.nbcnews.com/news/us-news/russian-ol...,https://media4.s-nbcnews.com/j/newscms/2018_14...
4,CBC News,"Welcome to The National, the flagship nightly ...",2018-06-28T01:07:49.89455Z,"{'id': 'cbc-news', 'name': 'CBC News'}",The National | CBC News,http://www.cbc.ca/news/the-national-live-1.418...,https://i.cbc.ca/1.4183530.1530144301!/httpIma...
5,CBC News,A girl who died in an apparent drowning along ...,2018-06-28T01:07:49.207Z,"{'id': 'cbc-news', 'name': 'CBC News'}",'Sombre day': 5-year-old who died at resort re...,http://www.cbc.ca/news/canada/toronto/collingw...,https://i.cbc.ca/1.4724844.1530122483!/fileIma...
6,Gulnaz Khan,The geographically isolated Tiwi people have p...,2018-06-28T01:07:21.6418017Z,"{'id': 'national-geographic', 'name': 'Nationa...",Dreamlike Photos of Australia's Unspoiled Para...,https://www.nationalgeographic.com/travel/dest...,https://www.nationalgeographic.com/content/dam...
7,Karen Gardiner,From the spectacular Eiffel Tower display to T...,2018-06-28T01:07:21.048004Z,"{'id': 'national-geographic', 'name': 'Nationa...",25 Dazzling Fireworks Pictures,https://www.nationalgeographic.com/travel/list...,https://www.nationalgeographic.com/content/dam...
8,Heather Brady,The group calling themselves 'Dare' says they ...,2018-06-28T01:07:20.6573482Z,"{'id': 'national-geographic', 'name': 'Nationa...","Amid a Civil War, These Syrians Keep Dancing",https://news.nationalgeographic.com/2018/06/da...,https://pmdvod.nationalgeographic.com/NG_Video...
9,Stephen Leahy,Brazil and the Democratic Republic of the Cong...,2018-06-28T01:07:18.7196952Z,"{'id': 'national-geographic', 'name': 'Nationa...",Tropical Forest Loss Slowed in 2017—To the Sec...,https://news.nationalgeographic.com/2018/06/tr...,https://news.nationalgeographic.com/content/da...


In [21]:
news_channel_name_list = [article['source']['name'] for article in headlines['articles']]

In [23]:
#news_channel_name_list

In [24]:
news_top_headlines_df['News Channel Name'] = news_channel_name_list

In [25]:
news_top_headlines_df

,author,description,publishedAt,source,title,url,urlToImage,News Channel Name
0,Yonah Jeremy Bob,HRW lawyer: Government not really after “the ‘...,2018-06-28T03:12:00Z,"{'id': 'the-jerusalem-post', 'name': 'The Jeru...",To deport HRW official or not? Court wrestles ...,https://www.jpost.com/Israel-News/To-deport-Hu...,"https://images.jpost.com/image/upload/f_auto,f...",The Jerusalem Post
1,Tovah Lazaroff,'As long as the politics and the obsessive pur...,2018-06-28T02:20:00Z,"{'id': 'the-jerusalem-post', 'name': 'The Jeru...",Israeli envoy: We should delay UNESCO exit,https://www.jpost.com/Arab-Israeli-Conflict/Is...,"https://images.jpost.com/image/upload/f_auto,f...",The Jerusalem Post
2,Fox News,President Trump praised retiring Supreme Court...,2018-06-28T01:15:00+00:00,"{'id': 'fox-news', 'name': 'Fox News'}",Trump says keeping Senate in GOP hands 'the mo...,http://www.foxnews.com/politics/2018/06/27/tru...,http://a57.foxnews.com/media2.foxnews.com/Brig...,Fox News
3,Reuters,An application for a warrant to search Paul Ma...,2018-06-28T01:07:57.9889393Z,"{'id': 'nbc-news', 'name': 'NBC News'}",Russian oligarch loaned $10 million to Manafor...,http://www.nbcnews.com/news/us-news/russian-ol...,https://media4.s-nbcnews.com/j/newscms/2018_14...,NBC News
4,CBC News,"Welcome to The National, the flagship nightly ...",2018-06-28T01:07:49.89455Z,"{'id': 'cbc-news', 'name': 'CBC News'}",The National | CBC News,http://www.cbc.ca/news/the-national-live-1.418...,https://i.cbc.ca/1.4183530.1530144301!/httpIma...,CBC News
5,CBC News,A girl who died in an apparent drowning along ...,2018-06-28T01:07:49.207Z,"{'id': 'cbc-news', 'name': 'CBC News'}",'Sombre day': 5-year-old who died at resort re...,http://www.cbc.ca/news/canada/toronto/collingw...,https://i.cbc.ca/1.4724844.1530122483!/fileIma...,CBC News
6,Gulnaz Khan,The geographically isolated Tiwi people have p...,2018-06-28T01:07:21.6418017Z,"{'id': 'national-geographic', 'name': 'Nationa...",Dreamlike Photos of Australia's Unspoiled Para...,https://www.nationalgeographic.com/travel/dest...,https://www.nationalgeographic.com/content/dam...,National Geographic
7,Karen Gardiner,From the spectacular Eiffel Tower display to T...,2018-06-28T01:07:21.048004Z,"{'id': 'national-geographic', 'name': 'Nationa...",25 Dazzling Fireworks Pictures,https://www.nationalgeographic.com/travel/list...,https://www.nationalgeographic.com/content/dam...,National Geographic
8,Heather Brady,The group calling themselves 'Dare' says they ...,2018-06-28T01:07:20.6573482Z,"{'id': 'national-geographic', 'name': 'Nationa...","Amid a Civil War, These Syrians Keep Dancing",https://news.nationalgeographic.com/2018/06/da...,https://pmdvod.nationalgeographic.com/NG_Video...,National Geographic
9,Stephen Leahy,Brazil and the Democratic Republic of the Cong...,2018-06-28T01:07:18.7196952Z,"{'id': 'national-geographic', 'name': 'Nationa...",Tropical Forest Loss Slowed in 2017—To the Sec...,https://news.nationalgeographic.com/2018/06/tr...,https://news.nationalgeographic.com/content/da...,National Geographic


In [ ]:
# remove the stopwords
remove_words = ['the', 'an', 'a', 'is', 'i', 'and', 'then', 'like', 'there', 'their']

In [ ]:
#remove the punctuation
def remove_punctuation(chars):
    return ''.join([c for c in chars if c not in punctuation])

# The DataFrame is 'news_top_headlines_df'
df = news_top_headlines_df

In [ ]:
#import our dataframe

df =

In [ ]:
#tokenize the text within the dataframe
df.loc[:, 'text_split'] = df.text.map(sent_tokenize)

In [ ]:
#get the text ready to analyze
sentences = []
for _, r in df.iterrows():
    for s in r("Headline").text_split:
        filtered_words = [remove_punctuation(w) for w in s.split() if w.lower() not in remove_words]
        # or using nltk.word_tokenize
        # filtered_words = [w for w in word_tokenize(s) if w.lower() not in remove_words and w not in punctuation]
        sentences.append({'sent_id': r.sent_id, 
                          'text': s.strip('.'), 
                          'words': filtered_words})
    wordfrq = r("Headline").text_split()

In [ ]:
#masterlist
masterlist = []

for channel in df:
    masterlist.append({'Channel': wordfrq})

In [ ]:
#rename the dataframe
df_words = pd.DataFrame(sentences)

In [ ]:
#word_cloud
#word freq
freq = nltk.FreqDist(wordfrq)
#tfidf
#stemming
#limitization
#thematic analysis

In [ ]:
#start the vader analysis
#run the vader on the orginal text
#import vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()